#### Test mynds packages in the sandbox

In [1]:
from collections.abc import Mapping
from functools import partial
from pathlib import Path
from typing import Callable, TypeVar

import tqdm

from mynd.image import (
    Image,
    ImageType,
    ImageComposite,
    ImageCompositeLoader,
)
from mynd.io import read_image

from mynd.utils.composition import create_composite_builder
from mynd.utils.filesystem import (
    list_directory,
    create_resource,
    Resource,
    ResourceManager,
    create_resource_matcher,
)
from mynd.utils.log import logger
from mynd.utils.result import Ok, Err, Result


IMAGE_PATTERN: str = "*.tiff"

Resources = list[Resource]


def collect_image_resources(directories: dict, tags: dict) -> ResourceManager:
    """Collects image resources and adds them to a manager."""
    image_manager: ResourceManager = ResourceManager()

    # Find file handles
    image_files: dict[ImageType, list[Path]] = {
        image_type: list_directory(directory, IMAGE_PATTERN)
        for image_type, directory in directories.items()
    }

    # Create resources out of the file handles
    for image_type, files in image_files.items():
        resources: Resources = [
            create_resource(path) for path in files if path.exists()
        ]

        if len(resources) == 0:
            logger.warning(f"no resources for type: {image_type}")

        image_manager.add_resources(resources, tags=tags.get(image_type))

    return image_manager


# NOTE: Defines how components are loaded
def create_image_composite_loader(
    components: dict[ImageType, Resource]
) -> ImageCompositeLoader:
    """Creates an image composite loader from component sources."""

    def load_image_composite() -> ImageComposite:
        """Loads an image composite from resources."""
        image_components: dict[ImageType, Image] = {
            image_type: read_image(resource.handle).unwrap()
            for image_type, resource in components.items()
        }
        return ImageComposite(image_components)

    return load_image_composite


# NOTE: Defines how components are labelled
def label_image_composite(components: Mapping[ImageType, Resource]) -> str:
    """Labels a collection of image components."""
    if ImageType.COLOR in components:
        label: str = components.get(ImageType.COLOR).stem
    else:
        label: str = [components.values()][0].stem
    return label


def log_image_queries(
    query_results: Mapping[ImageType, list[Resource]]
) -> None:
    """Log some image query statistics."""
    for category, resources in query_results.items():
        file_count: int = len(
            [resource for resource in resources if resource.is_file()]
        )
        dir_count: int = len(
            [resource for resource in resources if resource.is_directory()]
        )

        logger.info(f"Category: {category}, resources: {len(resources)}")
        logger.info(f" - Files:        {file_count}")
        logger.info(f" - Directories:  {dir_count}")


def main() -> None:
    """Main entrypoint."""

    DATADIR: Path = Path("/home/martin/data")

    directories: dict[str, Path] = {
        ImageType.COLOR: DATADIR
        / Path("acfr_images_debayered/r23685bc_20100605_021022_debayered"),
        ImageType.RANGE: DATADIR
        / Path("acfr_stereo_ranges/r23685bc_20100605_021022_ranges"),
        ImageType.NORMAL: DATADIR
        / Path("acfr_stereo_normals/r23685bc_20100605_021022_normals"),
    }

    tags: dict[ImageType, list[str]] = {
        ImageType.COLOR: ["image", "color"],
        ImageType.RANGE: ["image", "range"],
        ImageType.NORMAL: ["image", "normal"],
    }

    image_types: list[ImageType] = list(directories.keys())

    # Step 1 - Collect image resources
    image_manager: ResourceManager = collect_image_resources(directories, tags)

    # Step 2 - Retrieve image resources by type
    query_results: dict[ImageType, list[Resource]] = {
        image_type: image_manager.query_tags(tags.get(image_type))
        for image_type in image_types
    }

    log_image_queries(query_results)

    # Step 3 - Match image resources to create composites
    matcher = create_resource_matcher()
    matches: list[ComponentMatch] = matcher(query_results)

    # Step 4 - Create loaders from the matched resources
    builder = create_composite_builder(
        factory=create_image_composite_loader,
        labeller=label_image_composite,
    )

    loaders: dict[str, ImageCompositeLoader] = builder(matches)

    for label, loader in tqdm.tqdm(loaders.items(), desc="loading images..."):
        composite = loader()


# INVOKE MAIN
main()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/home/martin/.cache/pypoetry/virtualenvs/mynd-qzh2EXyG-py3.11/lib/python3.11/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
2024-10-10 15:01:32.682 | INFO     | __main__:log_image_queries:95 - Category: color, resources: 5458
2024-10-10 15:01:32.683 | INFO     | __main__:log_image_queries:96 -  - Files:        5458
2024-10-10 15:01:32.684 | INFO     | __main__:log_image_queries:97 -  - Directories:  0
2024-10-10 15:01:32.725 | INFO     | __main__:log_image_queries:95 - Category: range, resources: 5456
2024-10-10 15:01:32.726 | INFO     | __main__:log_image_queries:96 -  - Files:        5456
2024-10-10 15:01:32.726 | INFO     | __main__:log_image_queries:97 -  - Directories:  0
2024-10-10 15:01:32.778 | INFO     | __main__:log_image_queries:95 - Category: normal, resources: 5456
2024-10-10 15:01: